# Importaciones

In [1]:
import json 
import datetime
import re

import pandas as pd 
from googleapiclient.errors import HttpError
from Google import Create_Service

Función que utiliza expresiones regulares para obtener los segundos totales de una duración dada.

In [2]:
def convert_duration(duration):
    try:
        h = int(re.search('\d+H', duration)[0][:-1]) * 60**2  if re.search('\d+H', duration) else 0 # hour
        m = int(re.search('\d+M', duration)[0][:-1]) * 60  if re.search('\d+M', duration) else 0 # minute
        s = int(re.search('\d+S', duration)[0][:-1])  if re.search('\d+S', duration) else 0 # second
        return h + m + s
    except Exception as e:
        print(e)
        return 0 

In [3]:
def retrieve_playlists(service, partString, channelId):
    playlists =[]
    try:
        response = service.playlists().list(
            part = 'snippet',
            channelId = channelId,
            maxResults = 50
        ).execute()

        playlists.extend(response.get('items'))
        nextPageToken = response.get('nextPageToken')

        while nextPageToken:
            response = service.playlists().list(
                part = 'snippet',
                channelId = channelId,
                maxResults = 50,
                pageToken = nextPageToken
            ).execute()

            playlists.extend(response.get('items'))
            nextPageToken = response.get('nextPageToken')

    except HttpError as e:
        errMsg = json.loads(e.content)
        print("HTTP Error: ")
        print(errMsg['error']['message'])
        return []

Creando constantes para llamar a la API

In [2]:
CLIENT_SECRETS_FILE = 'client_secret.json'
API_NAME = 'youtube'
API_VERSION = 'v3'
SCOPES = ['https://www.googleapis.com/auth/youtube.readonly']

service = Create_Service(CLIENT_SECRETS_FILE, API_NAME, API_VERSION, SCOPES)

client_secret.json-youtube-v3-(['https://www.googleapis.com/auth/youtube.readonly'],)
['https://www.googleapis.com/auth/youtube.readonly']
Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=6670118319-vqkm9egmjnkmhdjm29manigl4490vpc3.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.readonly&state=3sSkwPy585upAZd6lDzt8FpAUZyDjw&access_type=offline
youtube service created successfully


El objeto <code>service</code> nos da accesos a todos los recursos de la API

In [4]:
dir(service)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_basic_methods',
 '_add_nested_resources',
 '_add_next_methods',
 '_baseUrl',
 '_developerKey',
 '_dynamic_attrs',
 '_http',
 '_model',
 '_requestBuilder',
 '_resourceDesc',
 '_rootDesc',
 '_schema',
 '_set_dynamic_attr',
 '_set_service_methods',
 'abuseReports',
 'activities',
 'captions',
 'channelBanners',
 'channelSections',
 'channels',
 'close',
 'commentThreads',
 'comments',
 'i18nLanguages',
 'i18nRegions',
 'liveBroadcasts',
 'liveChatBans',
 'liveChatMessages',
 'liveChatModerators',
 'liveStreams',
 'members',
 'membershipsLevels',
 'new_batch_h